In [4]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

def load_result_csv_files(directory):
    result_dfs = []
    file_paths = []
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith("results.csv"):
                file_path = os.path.join(root, filename)
                relative_path = os.path.relpath(file_path, directory)
                df = pd.read_csv(file_path)
                result_dfs.append(df)
                file_paths.append(relative_path)
    return result_dfs, file_paths

directory_path = "/workdir/optimal-summaries-public/_models_ablation"
result_dfs, file_paths = load_result_csv_files(directory_path)
len(result_dfs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


15

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [18]:
combined_df = pd.concat(result_dfs, ignore_index=True)
combined_df

,Model,Indicators,Summaries,Dataset,Seed,Split,AUC,ACC,F1,Cutoff,Lower threshold,Upper threshold
0,shared_encode_time_dim_True,True,True,tiselac,1,val,0.968943,0.759829,0.713257,"[0.36, 0.48222222222222216, 0.0933333333333333...","[-0.17, -0.44, -0.44, -0.53, -0.56, -1.17, -1....","[-0.17, -0.44, -0.44, -0.53, -0.56, -1.17, -1...."
1,shared_encode_time_dim_True,True,True,tiselac,1,test,0.966887,0.758930,0.709996,"[0.36, 0.48222222222222216, 0.0933333333333333...","[-0.17, -0.44, -0.44, -0.53, -0.56, -1.17, -1....","[-0.17, -0.44, -0.44, -0.53, -0.56, -1.17, -1...."
2,shared_encode_time_dim_True,True,True,tiselac,2,val,0.960508,0.740776,0.689594,"[0.4666666666666667, 0.4022222222222222, 0.139...","[-0.11, 0.69, -0.46, -0.56, -0.65, -1.87, 0.0,...","[-0.11, 0.69, -0.46, -0.56, -0.65, -1.87, 0.0,..."
3,shared_encode_time_dim_True,True,True,tiselac,2,test,0.964620,0.740879,0.690385,"[0.4666666666666667, 0.4022222222222222, 0.139...","[-0.11, 0.69, -0.46, -0.56, -0.65, -1.87, 0.0,...","[-0.11, 0.69, -0.46, -0.56, -0.65, -1.87, 0.0,..."
4,shared_encode_time_dim_True,True,True,tiselac,3,val,0.960635,0.732273,0.683252,"[0.27999999999999997, 0.2555555555555556, 0.09...","[-0.1, -0.47, -0.4, 0.0, -0.0, -0.41, 0.0, 1.3...","[-0.1, -0.47, -0.4, 0.0, -0.0, -0.41, 0.0, 1.3..."
5,shared_encode_time_dim_True,True,True,tiselac,3,test,0.960828,0.735377,0.683546,"[0.27999999999999997, 0.2555555555555556, 0.09...","[-0.1, -0.47, -0.4, 0.0, -0.0, -0.41, 0.0, 1.3...","[-0.1, -0.47, -0.4, 0.0, -0.0, -0.41, 0.0, 1.3..."
6,shared_encode_time_dim_True,False,True,tiselac,1,val,0.964106,0.744311,0.693223,"[0.45666666666666655, 0.27888888888888885, 0.1...","[0.01, -0.01, -0.45, -0.0, -0.67, -1.4, -0.2, ...","[0.01, -0.01, -0.45, -0.0, -0.67, -1.4, -0.2, ..."
7,shared_encode_time_dim_True,False,True,tiselac,1,test,0.961765,0.736325,0.686611,"[0.45666666666666655, 0.27888888888888885, 0.1...","[0.01, -0.01, -0.45, -0.0, -0.67, -1.4, -0.2, ...","[0.01, -0.01, -0.45, -0.0, -0.67, -1.4, -0.2, ..."
8,shared_encode_time_dim_True,False,True,tiselac,2,val,0.962133,0.746287,0.698508,"[0.5322222222222222, 0.7144444444444443, 0.173...","[0.29, -0.23, -0.4, 0.0, -0.03, -0.65, -0.65, ...","[0.29, -0.23, -0.4, 0.0, -0.03, -0.65, -0.65, ..."
9,shared_encode_time_dim_True,False,True,tiselac,2,test,0.965230,0.754964,0.705323,"[0.5322222222222222, 0.7144444444444443, 0.173...","[0.29, -0.23, -0.4, 0.0, -0.03, -0.65, -0.65, ...","[0.29, -0.23, -0.4, 0.0, -0.03, -0.65, -0.65, ..."


### Ablation


In [19]:
# Model,Indicators,Summaries,Dataset,Seed,Split,AUC,ACC,F1,Cutoff,Lower threshold,Upper threshold
combined_df.drop(columns=['Seed', "Cutoff", "Lower threshold", "Upper threshold"]).groupby(["Dataset", "Model", "Indicators", "Summaries", "Split"]).agg(["mean", "std"]).round(3)


AUC  \
                                                                               mean   
Dataset              Model                        Indicators Summaries Split          
mimic                atomics_sum2atomics_False    False      False     test   0.910   
                                                                       val    0.910   
                                                             True      test   0.868   
                                                                       val    0.869   
                                                  True       False     test   0.910   
                                                                       val    0.908   
                                                             True      test   0.877   
                                                                       val    0.879   
                     atomics_sum2atomics_True     False      False     test   0.908   
                                                                       val    0.908   
                                                             True      test   0.909   
                                                                       val    0.910   
                                                  True       False     test   0.907   
                                                                       val    0.908   
                                                             True      test   0.910   
                                                                       val    0.911   
                     original                     False      False     test   0.908   
                                                                       val    0.909   
                                                             True      test   0.875   
                                                                       val    0.875   
                                                  True       False     test   0.914   
                                                                       val    0.916   
                                                             True      test   0.869   
                                                                       val    0.870   
                     shared_encode_time_dim_False False      False     test   0.908   
                                                                       val    0.909   
                                                             True      test   0.913   
                                                                       val    0.916   
                                                  True       False     test   0.915   
                                                                       val    0.918   
                                                             True      test   0.918   
                                                                       val    0.921   
                     shared_encode_time_dim_True  False      False     test   0.905   
                                                                       val    0.908   
                                                             True      test   0.911   
                                                                       val    0.913   
                                                  True       False     test   0.916   
                                                                       val    0.918   
                                                             True      test   0.911   
                                                                       val    0.913   
spoken_arabic_digits atomics_sum2atomics_False    False      False     test   0.997   
                                                                       val    0.997   
                                                             True      test   0.505   
                                                                       val    

In [20]:
grouped = combined_df.drop(columns=["Seed", "Dataset", "Cutoff", "Lower threshold", "Upper threshold"]).groupby(["Model", "Indicators", "Summaries", "Split"]).agg(["mean", "std"]).round(3)#.reset_index()
# grouped = grouped.style.highlight_max(color = 'green', axis = 0).highlight_min(color = 'red', axis = 0)
grouped

# best auc 0.966000 acc 0.871000 f1 0.857000

AUC           ACC  \
                                                          mean    std   mean   
Model                        Indicators Summaries Split                        
atomics_sum2atomics_False    False      False     test   0.954  0.038  0.831   
                                                  val    0.954  0.038  0.834   
                                        True      test   0.725  0.213  0.489   
                                                  val    0.725  0.213  0.490   
                             True       False     test   0.954  0.038  0.833   
                                                  val    0.953  0.039  0.835   
                                        True      test   0.729  0.217  0.494   
                                                  val    0.729  0.217  0.494   
atomics_sum2atomics_True     False      False     test   0.954  0.039  0.830   
                                                  val    0.953  0.039  0.832   
                                        True      test   0.955  0.037  0.828   
                                                  val    0.955  0.037  0.828   
                             True       False     test   0.953  0.040  0.835   
                                                  val    0.953  0.039  0.837   
                                        True      test   0.951  0.034  0.808   
                                                  val    0.951  0.033  0.808   
original                     False      False     test   0.951  0.036  0.813   
                                                  val    0.951  0.036  0.814   
                                        True      test   0.691  0.201  0.436   
                                                  val    0.689  0.201  0.435   
                             True       False     test   0.952  0.034  0.813   
                                                  val    0.953  0.033  0.814   
                                        True      test   0.727  0.218  0.500   
                                                  val    0.727  0.218  0.500   
shared_encode_time_dim_False False      False     test   0.956  0.040  0.847   
                                                  val    0.956  0.039  0.849   
                                        True      test   0.961  0.038  0.862   
                                                  val    0.962  0.036  0.862   
                             True       False     test   0.958  0.036  0.851   
                                                  val    0.959  0.035  0.852   
                                        True      test   0.963  0.036  0.865   
                                                  val    0.963  0.034  0.865   
shared_encode_time_dim_True  False      False     test   0.955  0.041  0.844   
                                                  val    0.956  0.040  0.847   
                                        True      test   0.796  0.214  0.564   
                                                  val    0.795  0.215  0.565   
                             True       False     test   0.958  0.036  0.852   
                                                  val    0.959  0.035  0.851   
                                        True      test   0.796  0.213  0.565   
                                                  val    0.796  0.215  0.565   

                                                                   F1         
                                                           std   mean    std  
Model                        Indicators Summaries Split                       
atomics_sum2atomics_False    False      False     test   0.096  0.820  0.115  
                                                  val    0.097  0.823  0.115  
                                        True      test   0.367  0.444  0.404  
                                                  val    0.368  0.445  0.406  
                             True       False     test   0.102  0